# Trading agent demo

## Imports

In [13]:
from modules.data_loader import DataLoader
from modules.single_asset_env import SingleAssetEnv
from modules.q_network import Q_network
from modules.memory import Memory
from modules.trading_agent import TradingAgent

import torch
import numpy as np

## Environment variables

In [14]:
NUM_ACTIONS = 3 # Buy, Sell, Hold
LEN_HISTORY = 7 # consider last week to predict next value
STATES_DIM = LEN_HISTORY + 1 # history + predicted value

# Q network params
INPUT_DIM = STATES_DIM
HIDDEN_DIM = 64
OUTPUT_DIM = NUM_ACTIONS

LEARNING_RATE = 1e-3

NUM_EPOCHS = 20
BATCH_SIZE = 64

MEMORY_SIZE = 200

GAMMA = 0.97

EPSILON = 1.0
EPSILON_DECREASE = 1e-3
EPSILON_MIN = 0.1
START_REDUCE_EPSILON = 200

TRAIN_FREQ = 10
UPDATE_Q_FREQ = 20
SHOW_LOG_FREQ = 5

DEVICE = 'cpu'
SEED = 123

## Loading data

In [27]:
dataloader = DataLoader('2020-01-01', '1d', '2024-01-01')

assets = ["BTC-USD", "ETH-USD", "BNB-USD"]

trains = []
tests = []

for asset in assets:
    train, test = dataloader.load(asset)

    trains.append(train)
    tests.append(test)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## Initialisations

In [16]:
train_envs = [SingleAssetEnv(train) for train in trains]

agent = TradingAgent(STATES_DIM, NUM_ACTIONS, seed=SEED)

scores = {key: [] for key in assets}

## Training the agent

In [17]:
for epoch in range(NUM_EPOCHS):

    # intialise score for the epoch
    score = {key: 0 for key in assets}
    step_count = 1

    for asset, env in zip(assets, train_envs):

        # reset the environment before each epoch + get initial state
        state = env.reset()

        while True:

            # find epsilon greedy action from state
            action = agent.act(state, 1/step_count) # epsilon = 1/t

            # perform step in the environment and get completing info
            next_state, reward, done = env.step(action)

            agent.step(state, action, reward, next_state, done)

            # prepare for next iteration
            step_count += 1
            state = next_state

            score[asset] += reward

            if done:
                break

    # compute info about the epoch
    for key in scores.keys():
        scores[key].append(score[key])

    print(f"Epoch {epoch:2} | Score = {score}")

print("Training done!")

# save Q_network model weights
agent.save_models("weights")

Epoch  0 | Score = {'BTC-USD': -8850736.239257812, 'ETH-USD': -1462, 'BNB-USD': -1462}
Epoch  1 | Score = {'BTC-USD': -203924.23828125, 'ETH-USD': -1462, 'BNB-USD': -1462}
Epoch  2 | Score = {'BTC-USD': -139537.455078125, 'ETH-USD': -101461, 'BNB-USD': -1462}
Epoch  3 | Score = {'BTC-USD': -3024421.5849609375, 'ETH-USD': -1462, 'BNB-USD': -1682.680435180664}
Epoch  4 | Score = {'BTC-USD': -158477.3671875, 'ETH-USD': -1462, 'BNB-USD': -1462}
Epoch  5 | Score = {'BTC-USD': -301671.7041015625, 'ETH-USD': -2235.5349731445312, 'BNB-USD': -1462}
Epoch  6 | Score = {'BTC-USD': -188265.64599609375, 'ETH-USD': -101461, 'BNB-USD': -101461}
Epoch  7 | Score = {'BTC-USD': 0.29052734375, 'ETH-USD': -2906.3834228515625, 'BNB-USD': -1783.70751953125}
Epoch  8 | Score = {'BTC-USD': -953353.6010742188, 'ETH-USD': -1462, 'BNB-USD': -101461}
Epoch  9 | Score = {'BTC-USD': -171272.50634765625, 'ETH-USD': -4879.408203125, 'BNB-USD': -1462}
Epoch 10 | Score = {'BTC-USD': -198583.361328125, 'ETH-USD': -20146

## Testing the agent

In [33]:
test_envs = [SingleAssetEnv(test) for test in tests]

for asset, env in zip(assets, test_envs):

    state = env.reset()

    test_actions = []
    test_rewards = []

    for _ in range(len(env.data)-1):
        
        action = agent.qnet(torch.from_numpy(np.array(state, dtype=np.float32).reshape(1, -1)))
        action = np.argmax(action.data)
        test_actions.append(action.item())
                
        next_state, reward, done = env.step(action.numpy())
        test_rewards.append(reward)

        state = next_state

    print(f"Profit made with {asset}: {env.profit}")

print("-"*27)
print(f"Total profit made: {sum([env.profit for env in test_envs])}")

Profit made with BTC-USD: 0
Profit made with ETH-USD: 0
Profit made with BNB-USD: 0
---------------------------
Total profit made: 0
